# LangGraph Human-in-the-Loop Lab
## Comprehensive Hands-on Lab Exercises

This notebook contains 4 complete lab exercises covering all human-in-the-loop patterns:

1. **Lab 1**: Loan Approval with Static Interrupts (`interrupt_before`)
2. **Lab 2**: Fraud Detection with Dynamic Interrupts (`interrupt()` function)
3. **Lab 3**: Account Opening with State Editing (`update_state()`)
4. **Lab 4**: Customer Support with Multi-turn Conversations


### Setup
Set your GROQ_API_KEY environment variable before running the labs.

In [1]:
# Setup: Configure your GROQ API Key
import os
from getpass import getpass
from dotenv import load_dotenv

# Load environment variables from .env
load_dotenv()

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass("Enter your GROQ API Key: ")

print("✅ API Key configured")


✅ API Key configured


---
## Lab 1: Loan Approval with Static Interrupts

### Learning Objectives:
- Understand static interrupts using `interrupt_before` and `interrupt_after`
- Learn checkpoint-based persistence
- Practice reviewing and approving actions before execution

### Scenario:
A loan application system where a human loan officer must review AI-generated risk assessments before final loan decisions are made.

In [2]:
# Lab 1: Imports
import os
from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

# Define State
class LoanApplicationState(TypedDict):
    applicant_name: str
    loan_amount: float
    annual_income: float
    credit_score: int
    employment_years: int
    risk_assessment: str
    loan_decision: str
    officer_notes: str
    messages: Annotated[list, "conversation history"]

# Initialize LLM
def get_llm():
    return ChatGroq(
        model="llama-3.1-8b-instant",
    )

print("✅ Lab 1 setup complete")

✅ Lab 1 setup complete


In [3]:
# Lab 1: Define Workflow Nodes

def analyze_loan_application(state: LoanApplicationState) -> LoanApplicationState:
    """AI analyzes the loan application and provides risk assessment"""
    llm = get_llm()
    
    system_message = """You are a financial risk analyst. 
    Analyze the loan application and provide a comprehensive risk assessment.
    Consider: credit score, debt-to-income ratio, employment stability.
    
    Risk Categories:
    - LOW RISK: Credit score > 750, 5+ years employment, loan < 30% income
    - MEDIUM RISK: Credit score 650-750, 2-5 years employment, loan 30-50% income  
    - HIGH RISK: Credit score < 650, < 2 years employment, loan > 50% income"""
    
    prompt = f"""
    Loan Application:
    - Applicant: {state['applicant_name']}
    - Loan Amount: ${state['loan_amount']:,.2f}
    - Annual Income: ${state['annual_income']:,.2f}
    - Credit Score: {state['credit_score']}
    - Years of Employment: {state['employment_years']}
    
    Provide your risk assessment and recommendation.
    """
    
    messages = [SystemMessage(content=system_message), HumanMessage(content=prompt)]
    response = llm.invoke(messages)
    
    print("\n" + "="*80)
    print("🤖 AI RISK ASSESSMENT")
    print("="*80)
    print(response.content)
    print("="*80)
    
    return {
        "risk_assessment": response.content,
        "messages": state.get("messages", []) + [HumanMessage(content=prompt), AIMessage(content=response.content)]
    }

def make_loan_decision(state: LoanApplicationState) -> LoanApplicationState:
    """Make final loan decision based on risk assessment"""
    llm = get_llm()
    
    prompt = f"""
    Make final decision for: {state['applicant_name']}
    Loan Amount: ${state['loan_amount']:,.2f}
    
    Risk Assessment: {state['risk_assessment']}
    Officer Notes: {state.get('officer_notes', 'None')}
    
    Provide: APPROVED or DENIED with terms and reasoning.
    """
    
    response = llm.invoke([HumanMessage(content=prompt)])
    
    print("\n" + "="*80)
    print("✅ FINAL LOAN DECISION")
    print("="*80)
    print(response.content)
    print("="*80)
    
    return {"loan_decision": response.content}

print("✅ Nodes defined")

✅ Nodes defined


In [4]:
# Lab 1: Build and Compile Graph

def create_loan_workflow():
    workflow = StateGraph(LoanApplicationState)
    
    # Add nodes
    workflow.add_node("analyze_application", analyze_loan_application)
    workflow.add_node("make_decision", make_loan_decision)
    
    # Define edges
    workflow.add_edge(START, "analyze_application")
    workflow.add_edge("analyze_application", "make_decision")
    workflow.add_edge("make_decision", END)
    
    # Compile with STATIC INTERRUPT
    memory = MemorySaver()
    app = workflow.compile(
        checkpointer=memory,
        interrupt_before=["make_decision"]  # Pause before final decision
    )
    
    return app

print("✅ Workflow created")

✅ Workflow created


In [5]:
# Lab 1: Run Demo

app = create_loan_workflow()
thread_config = {"configurable": {"thread_id": "loan_001"}}

# Initial application
initial_state = {
    "applicant_name": "Sarah Martinez",
    "loan_amount": 250000.0,
    "annual_income": 85000.0,
    "credit_score": 720,
    "employment_years": 4,
    "risk_assessment": "",
    "loan_decision": "",
    "officer_notes": "",
    "messages": []
}

print("📋 Loan Application Submitted")
print(f"Applicant: {initial_state['applicant_name']}")
print(f"Loan Amount: ${initial_state['loan_amount']:,.2f}")
print(f"Credit Score: {initial_state['credit_score']}\n")

# Run until interrupt
print("🔄 Running workflow...\n")
for event in app.stream(initial_state, thread_config, stream_mode="values"):
    pass

# Check state
current_state = app.get_state(thread_config)
print("\n⏸️  INTERRUPTED - Awaiting Loan Officer Review")
print(f"Next Node: {current_state.next}")
print(f"Risk Assessment Generated: {'Yes' if current_state.values.get('risk_assessment') else 'No'}")

📋 Loan Application Submitted
Applicant: Sarah Martinez
Loan Amount: $250,000.00
Credit Score: 720

🔄 Running workflow...


🤖 AI RISK ASSESSMENT
**Risk Assessment:**

Based on the provided loan application information, I will evaluate Sarah Martinez's creditworthiness and assess the risk associated with granting her the loan.

**Credit Score:** 720
This credit score is above the threshold for medium risk, indicating a strong history of credit management and a lower likelihood of default.

**Annual Income:** $85,000.00
With an annual income of $85,000.00, Sarah's loan-to-income ratio is calculated as follows:

Loan Amount / Annual Income = $250,000.00 / $85,000.00 ≈ 2.94 (or approximately 294%)

This ratio exceeds the 30% threshold for medium risk, indicating a higher risk of default due to a significant portion of her income being dedicated to loan repayment.

**Years of Employment:** 4
Sarah has been employed for 4 years, which falls within the 2-5 year range for medium risk. This indi

In [6]:
# Lab 1: Simulate Human Review and Resume

# Loan officer adds notes
officer_notes = """
Approved with conditions:
- Strong income and good credit score
- Solid employment history (4 years)
- Recommend standard rate with 20% down payment
"""

print("👤 LOAN OFFICER REVIEW")
print("="*80)
print(officer_notes)
print("="*80)

# Update state with officer notes
app.update_state(thread_config, {"officer_notes": officer_notes})

# Resume workflow
print("\n▶️  Resuming workflow...\n")
for event in app.stream(None, thread_config, stream_mode="values"):
    pass

print("\n✅ WORKFLOW COMPLETED")

👤 LOAN OFFICER REVIEW

Approved with conditions:
- Strong income and good credit score
- Solid employment history (4 years)
- Recommend standard rate with 20% down payment


▶️  Resuming workflow...


✅ FINAL LOAN DECISION
APPROVED with conditions.

Based on the officer's notes, the following conditions have been met:

- Strong income and good credit score: Sarah's credit score of 720 and annual income of $85,000.00 indicate a strong credit history and stable income.
- Solid employment history (4 years): Sarah's 4 years of employment history indicate a moderate level of employment stability.
- Recommend standard rate with 20% down payment: This condition suggests that the lender is willing to approve the loan with favorable terms, including a standard interest rate and a 20% down payment, which reduces the borrower's loan-to-value ratio.

Considering these conditions, the loan application has been approved with the following terms:

* Loan Amount: $250,000.00
* Down Payment: 20% ($50,0

### Lab 1 Exercises:
1. Change interrupt before to `interrupt_after=["analyze_application"]`
2. Add a compliance check node with another interrupt. Sample checks - Regulatory compliance (loan limits, borrower eligibility), Anti-money laundering (AML) requirements,    3. Fair lending practices, Documentation completeness. Shall interrupt before compliance check and before final decision.
3. Test with high-risk applicant (credit score < 650)
4. Implement rejection handling logic. Use conditional edges. Generate loan rejection letter.

---
## Lab 2: Fraud Detection with Dynamic Interrupts

### Learning Objectives:
- Use dynamic interrupts with `interrupt()` function
- Implement conditional human-in-the-loop
- Use `Command(resume=...)` to provide input

### Banking Scenario:
Automated fraud detection that only pauses for human review when high fraud risk is detected.

In [7]:
# Lab 2: Imports and Setup
from langgraph.types import interrupt, Command
import json

class TransactionState(TypedDict):
    transaction_id: str
    account_holder: str
    transaction_amount: float
    merchant: str
    location: str
    transaction_type: str
    account_balance: float
    recent_transactions: list
    fraud_score: float
    fraud_reason: str
    fraud_analyst_decision: str
    final_status: str
    messages: Annotated[list, "conversation history"]

print("✅ Lab 2 setup complete")

✅ Lab 2 setup complete


In [8]:
# Lab 2: Define Nodes with Dynamic Interrupt

def analyze_transaction(state: TransactionState) -> TransactionState:
    """Analyze transaction for fraud indicators"""
    llm = get_llm()
    
    system_message = """You are a fraud detection AI.
    Analyze transactions and calculate fraud risk score (0-100).
    High-risk: Large amounts (>$5,000), foreign locations, unusual patterns.
    Return ONLY JSON: {"fraud_score": <number>, "fraud_reason": "<text>"}"""
    
    prompt = f"""
    Transaction: {state['transaction_id']}
    Amount: ${state['transaction_amount']:,.2f}
    Merchant: {state['merchant']}
    Location: {state['location']}
    Balance: ${state['account_balance']:,.2f}
    
    Analyze and return JSON.
    """
    
    response = llm.invoke([SystemMessage(content=system_message), HumanMessage(content=prompt)])
    
    try:
        content = response.content.strip()
        if "```json" in content:
            content = content.split("```json")[1].split("```")[0].strip()
        fraud_data = json.loads(content)
        fraud_score = fraud_data.get('fraud_score', 30)
        fraud_reason = fraud_data.get('fraud_reason', 'Analysis completed')
    except:
        fraud_score = 30
        fraud_reason = "Error parsing response"
    
    print(f"\n🔍 Fraud Score: {fraud_score}/100")
    print(f"Reason: {fraud_reason}\n")
    
    return {"fraud_score": fraud_score, "fraud_reason": fraud_reason}

def review_suspicious_transaction(state: TransactionState) -> TransactionState:
    """DYNAMIC INTERRUPT: Only pause if fraud score > 60"""
    fraud_score = state['fraud_score']
    
    if fraud_score > 60:
        print(f"\n⚠️  HIGH FRAUD RISK ({fraud_score}) - Pausing for review")
        
        # DYNAMIC INTERRUPT
        analyst_decision = interrupt({
            "message": "Fraud analyst review required",
            "fraud_score": fraud_score,
            "amount": state['transaction_amount']
        })
        
        print(f"\n👤 Analyst Decision: {analyst_decision}\n")
        return {"fraud_analyst_decision": analyst_decision, "final_status": analyst_decision}
    else:
        print(f"\n✅ LOW RISK ({fraud_score}) - Auto-approved\n")
        return {"fraud_analyst_decision": "AUTO_APPROVED", "final_status": "APPROVED"}

def process_transaction(state: TransactionState) -> TransactionState:
    """Process transaction based on decision"""
    status = state['final_status']
    print(f"\n💳 Transaction {status}\n")
    return {"final_status": f"Transaction {state['transaction_id']} {status}"}

print("✅ Nodes defined")

✅ Nodes defined


In [9]:
# Lab 2: Build Workflow

def create_fraud_workflow():
    workflow = StateGraph(TransactionState)
    
    workflow.add_node("analyze_transaction", analyze_transaction)
    workflow.add_node("review_suspicious", review_suspicious_transaction)
    workflow.add_node("process_transaction", process_transaction)
    
    workflow.add_edge(START, "analyze_transaction")
    workflow.add_edge("analyze_transaction", "review_suspicious")
    workflow.add_edge("review_suspicious", "process_transaction")
    workflow.add_edge("process_transaction", END)
    
    memory = MemorySaver()
    app = workflow.compile(checkpointer=memory)
    
    return app

print("✅ Workflow created")

✅ Workflow created


In [10]:
# Lab 2: Test Suspicious Transaction

app = create_fraud_workflow()
thread_1 = {"configurable": {"thread_id": "txn_001"}}

suspicious_txn = {
    "transaction_id": "TXN-001",
    "account_holder": "John Smith",
    "transaction_amount": 8500.00,
    "merchant": "Electronics - Moscow",
    "location": "Moscow, Russia",
    "transaction_type": "Purchase",
    "account_balance": 12000.00,
    "recent_transactions": [{"amount": 45, "merchant": "Coffee"}],
    "fraud_score": 0.0,
    "fraud_reason": "",
    "fraud_analyst_decision": "",
    "final_status": "",
    "messages": []
}

print("="*80)
print("SCENARIO 1: Suspicious Transaction")
print("="*80)

for event in app.stream(suspicious_txn, thread_1, stream_mode="values"):
    pass

# Check if interrupted
state = app.get_state(thread_1)
if state.tasks:
    print("⏸️  Paused for analyst review")
    print(f"Interrupt data: {state.tasks[0].interrupts[0].value}")

SCENARIO 1: Suspicious Transaction

🔍 Fraud Score: 95/100
Reason: High-risk transaction due to large amount ($8,500.00) and foreign location (Moscow, Russia)


⚠️  HIGH FRAUD RISK (95) - Pausing for review
⏸️  Paused for analyst review
Interrupt data: {'message': 'Fraud analyst review required', 'fraud_score': 95, 'amount': 8500.0}


In [11]:
# Lab 2: Resume with Analyst Decision

print("\n👤 Fraud Analyst approves after verification\n")

# Resume with Command
for event in app.stream(Command(resume="APPROVED"), thread_1, stream_mode="values"):
    pass

print("✅ Transaction processed")


👤 Fraud Analyst approves after verification


⚠️  HIGH FRAUD RISK (95) - Pausing for review

👤 Analyst Decision: APPROVED


💳 Transaction APPROVED

✅ Transaction processed


In [12]:
# Lab 2: Test Normal Transaction (No Interrupt)

thread_2 = {"configurable": {"thread_id": "txn_002"}}

normal_txn = {
    "transaction_id": "TXN-002",
    "account_holder": "Jane Doe",
    "transaction_amount": 85.50,
    "merchant": "Grocery Store",
    "location": "New York, USA",
    "transaction_type": "Purchase",
    "account_balance": 5500.00,
    "recent_transactions": [{"amount": 42, "merchant": "Restaurant"}],
    "fraud_score": 0.0,
    "fraud_reason": "",
    "fraud_analyst_decision": "",
    "final_status": "",
    "messages": []
}

print("\n" + "="*80)
print("SCENARIO 2: Normal Transaction")
print("="*80)

for event in app.stream(normal_txn, thread_2, stream_mode="values"):
    pass

print("✅ Completed without human intervention")


SCENARIO 2: Normal Transaction

🔍 Fraud Score: 0/100
Reason: Low-risk transaction: amount is below the threshold and located within the domestic region.


✅ LOW RISK (0) - Auto-approved


💳 Transaction APPROVED

✅ Completed without human intervention


### Lab 2 Exercises:
1. Lower fraud threshold to 50 to see more interrupts
2. Add second interrupt for amounts > $10,000
3. Implement three-tier review system
4. Add rejection reason capture with another interrupt

---
## Lab 3: Account Opening Workflow with State Editing

### Learning Objectives:
- Edit Graph State pattern
- Using update_state() to modify values
- Reviewing and correcting AI-extracted information
- Multi-step validation workflow

### Scenario:
New account opening process where a bank representative can review, edit, and correct customer information before final submission.

In [ ]:
# Lab 3: Setup
class AccountOpeningState(TypedDict):
    """State for account opening workflow"""
    # Raw customer data
    raw_application: str
    
    # Extracted structured data
    customer_name: str
    date_of_birth: str
    ssn_last_four: str
    address: str
    phone: str
    email: str
    initial_deposit: float
    account_type: str
    
    # Validation flags
    data_validated: bool
    validation_errors: list
    
    # Final processing
    account_number: str
    final_status: str
    messages: Annotated[list, "conversation history"]
    
print("✅ Lab 3 setup complete")

In [ ]:
# Lab 3: Define Nodes
def extract_customer_data(state: AccountOpeningState) -> AccountOpeningState:
    """AI extracts structured data from raw application"""
    llm = get_llm()
    
    system_message = """You are a data extraction AI for a bank's account opening system.
    Extract customer information from the raw application text.
    
    Return ONLY a JSON object with these fields:
    - customer_name
    - date_of_birth (YYYY-MM-DD format)
    - ssn_last_four (last 4 digits only)
    - address (full street address)
    - phone (format: XXX-XXX-XXXX)
    - email
    - initial_deposit (numeric value)
    - account_type (checking or savings)
    
    If any field is missing or unclear, use "NEEDS_REVIEW" as the value.
    """
    
    prompt = f"""
    Extract customer data from this account application:
    
    {state['raw_application']}
    
    Return properly formatted JSON.
    """
    
    messages = [
        SystemMessage(content=system_message),
        HumanMessage(content=prompt)
    ]
    
    response = llm.invoke(messages)
    
    # Parse JSON response
    try:
        content = response.content.strip()
        if "```json" in content:
            content = content.split("```json")[1].split("```")[0].strip()
        elif "```" in content:
            content = content.split("```")[1].split("```")[0].strip()
        
        customer_data = json.loads(content)
    except Exception as e:
        print(f"Error parsing JSON: {e}")
        customer_data = {
            "customer_name": "NEEDS_REVIEW",
            "date_of_birth": "NEEDS_REVIEW",
            "ssn_last_four": "NEEDS_REVIEW",
            "address": "NEEDS_REVIEW",
            "phone": "NEEDS_REVIEW",
            "email": "NEEDS_REVIEW",
            "initial_deposit": 0.0,
            "account_type": "NEEDS_REVIEW"
        }
    
    print("\n" + "="*80)
    print("🤖 AI DATA EXTRACTION COMPLETED")
    print("="*80)
    print(json.dumps(customer_data, indent=2))
    print("="*80 + "\n")
    
    return {
        "customer_name": customer_data.get("customer_name", "NEEDS_REVIEW"),
        "date_of_birth": customer_data.get("date_of_birth", "NEEDS_REVIEW"),
        "ssn_last_four": customer_data.get("ssn_last_four", "NEEDS_REVIEW"),
        "address": customer_data.get("address", "NEEDS_REVIEW"),
        "phone": customer_data.get("phone", "NEEDS_REVIEW"),
        "email": customer_data.get("email", "NEEDS_REVIEW"),
        "initial_deposit": float(customer_data.get("initial_deposit", 0)),
        "account_type": customer_data.get("account_type", "NEEDS_REVIEW"),
        "messages": state.get("messages", []) + [
            HumanMessage(content=prompt),
            AIMessage(content=response.content)
        ]
    }


def validate_customer_data(state: AccountOpeningState) -> AccountOpeningState:
    """Validate extracted customer data"""
    errors = []
    
    # Validation rules
    if "NEEDS_REVIEW" in state.get('customer_name', ''):
        errors.append("Customer name needs review")
    
    if "NEEDS_REVIEW" in state.get('date_of_birth', ''):
        errors.append("Date of birth needs review")
    
    if state.get('ssn_last_four', '') == "NEEDS_REVIEW" or len(state.get('ssn_last_four', '')) != 4:
        errors.append("SSN last four digits invalid")
    
    if "NEEDS_REVIEW" in state.get('address', ''):
        errors.append("Address needs review")
    
    if "NEEDS_REVIEW" in state.get('email', '') or '@' not in state.get('email', ''):
        errors.append("Email address invalid")
    
    if state.get('initial_deposit', 0) < 100:
        errors.append("Initial deposit must be at least $100")
    
    if state.get('account_type', '') not in ['checking', 'savings']:
        errors.append("Account type must be checking or savings")
    
    is_valid = len(errors) == 0
    
    print("\n" + "="*80)
    print("✓ DATA VALIDATION")
    print("="*80)
    if is_valid:
        print("✅ All data validated successfully")
    else:
        print("⚠️  Validation errors found:")
        for error in errors:
            print(f"   - {error}")
    print("="*80 + "\n")
    
    return {
        "data_validated": is_valid,
        "validation_errors": errors
    }


def open_account(state: AccountOpeningState) -> AccountOpeningState:
    """Open the account and generate account number"""
    import random
    
    # Generate account number
    account_number = f"ACC-{random.randint(10000000, 99999999)}"
    
    print("\n" + "="*80)
    print("✅ ACCOUNT SUCCESSFULLY OPENED")
    print("="*80)
    print(f"Account Holder: {state['customer_name']}")
    print(f"Account Number: {account_number}")
    print(f"Account Type: {state['account_type'].upper()}")
    print(f"Initial Deposit: ${state['initial_deposit']:,.2f}")
    print("="*80 + "\n")
    
    return {
        "account_number": account_number,
        "final_status": "ACCOUNT_OPENED"
    }

print("✅ Nodes defined")

In [ ]:
# Lab 3: Build Workflow

def create_account_opening_workflow():
    """Create the account opening workflow"""
    workflow = StateGraph(AccountOpeningState)
    
    # Add nodes
    workflow.add_node("extract_data", extract_customer_data)
    workflow.add_node("validate_data", validate_customer_data)
    workflow.add_node("open_account", open_account)
    
    # Define edges
    workflow.add_edge(START, "extract_data")
    workflow.add_edge("extract_data", "validate_data")
    workflow.add_edge("validate_data", "open_account")
    workflow.add_edge("open_account", END)
    
    # Compile with checkpointer and interrupt after validation
    memory = MemorySaver()
    app = workflow.compile(
        checkpointer=memory,
        interrupt_after=["validate_data"]  # Pause after validation for human review
    )
    
    return app

print("✅ Workflow created")

In [ ]:
# Lab 3: Account opening with state editing

"""Demonstrate account opening with state editing"""

print("\n" + "="*80)
print("ACCOUNT OPENING WORKFLOW - STATE EDITING")
print("="*80 + "\n")

app = create_account_opening_workflow()

thread_config = {"configurable": {"thread_id": "account_open_001"}}

# Raw application (with some intentionally unclear data)
initial_state = {
    "raw_application": """
    New Account Application
    
    I would like to open a checking account. My name is Michael Thompson
    and I was born on March 15, 1985. The last four digits of my social 
    security number are 7392. 
    
    I live at 742 Evergreen Terrace, Springfield, IL 62701.
    
    You can reach me at michael.t@email.com or call me at 555-0123.
    
    I would like to make an initial deposit of $2,500.
    """,
    "customer_name": "",
    "date_of_birth": "",
    "ssn_last_four": "",
    "address": "",
    "phone": "",
    "email": "",
    "initial_deposit": 0.0,
    "account_type": "",
    "data_validated": False,
    "validation_errors": [],
    "account_number": "",
    "final_status": "",
    "messages": []
}

print("📄 RAW APPLICATION RECEIVED")
print("="*80)
print(initial_state['raw_application'])
print("="*80 + "\n")

# Run until interrupt
print("🔄 Processing application...\n")
for event in app.stream(initial_state, thread_config, stream_mode="values"):
    pass

# Check state at interrupt
print("\n⏸️  WORKFLOW PAUSED - Awaiting Bank Representative Review\n")
current_state = app.get_state(thread_config)

print("📊 CURRENT STATE FOR REVIEW:")
print("="*80)
state_values = current_state.values
print(f"Customer Name: {state_values.get('customer_name')}")
print(f"Date of Birth: {state_values.get('date_of_birth')}")
print(f"SSN (last 4): {state_values.get('ssn_last_four')}")
print(f"Address: {state_values.get('address')}")
print(f"Phone: {state_values.get('phone')}")
print(f"Email: {state_values.get('email')}")
print(f"Initial Deposit: ${state_values.get('initial_deposit', 0):,.2f}")
print(f"Account Type: {state_values.get('account_type')}")
print(f"\nData Validated: {state_values.get('data_validated')}")
if state_values.get('validation_errors'):
    print("Validation Errors:")
    for error in state_values['validation_errors']:
        print(f"   - {error}")
print("="*80 + "\n")

In [ ]:
# Lab 3: SIMULATE HUMAN REVIEW AND STATE EDITING

print("👤 BANK REPRESENTATIVE REVIEW")
print("="*80)
print("Representative identified issues:")
print("   - Phone number incomplete (missing area code)")
print("   - Need to correct phone to proper format")
print("\nEditing state to fix phone number...")
print("="*80 + "\n")

# EDIT THE STATE - Fix the phone number
app.update_state(
    thread_config,
    {
        "phone": "555-555-0123",  # Corrected phone number
        "data_validated": True,    # Mark as validated
        "validation_errors": []    # Clear errors
    }
)

print("✅ State updated with corrections\n")

# Verify the update
updated_state = app.get_state(thread_config)
print("📊 UPDATED STATE:")
print("="*80)
print(f"Phone: {updated_state.values.get('phone')} ✅ CORRECTED")
print(f"Data Validated: {updated_state.values.get('data_validated')}")
print("="*80 + "\n")

In [ ]:
# Lab 3: Resume workflow
print("▶️  RESUMING WORKFLOW - Opening Account...\n")
for event in app.stream(None, thread_config, stream_mode="values"):
    pass

# Get final state
final_state = app.get_state(thread_config)
print("✅ WORKFLOW COMPLETED")
print(f"Account Number: {final_state.values.get('account_number')}")
print(f"Status: {final_state.values.get('final_status')}\n")

### Lab 3 Exercises:
1. EDIT MULTIPLE FIELDS:
   - Modify the demo to correct multiple fields at once
   - Update customer name, address, and email simultaneously
   - Observe how update_state() merges changes
       
2. ADD APPROVAL STEP:
   - Add a new node "manager_approval" after validation
   - Use interrupt_before=["manager_approval"]
   - Let manager edit and approve/reject via state updates
   
3. IMPLEMENT CORRECTION LOOP:
   - Add conditional edge from validate_data
   - If validation fails, route back to extract_data
   - Allow up to 3 correction attempts
   
4. CREATE VALIDATION HISTORY:
   - Add "validation_history" list to state
   - Record each validation attempt with timestamp
   - Show full correction history before final approval
   
5. BUILD REVIEW INTERFACE:
   - Create a function that displays all fields for review
   - Prompt for which field to edit
   - Apply the edit using update_state()
   - Repeat until representative approves
   
6. ADD COMPLIANCE CHECKS:
   - Create a new node for compliance validation
   - Check age (must be 18+)
   - Check deposit amount limits
   - Pause for compliance officer review if issues found
   
7. TEST INCOMPLETE APPLICATIONS:
   - Create applications with missing required fields
   - See how AI marks them as "NEEDS_REVIEW"
   - Practice editing the state to fill in missing data

---
## Lab 4: Customer Support Agent with Multi-turn Conversations

### Learning Objectives:
- Multi-turn conversation pattern
- Gathering input over multiple interactions
- Command for managing conversation state
- Escalation to human agents
- Persistent conversation history

### Scenario:
An AI customer support agent that can have multi-turn conversations with customers, gathering information incrementally and escalating to human agents when needed.

In [ ]:
# Lab 4 import and setup

import operator
from langchain_core.messages import (
    HumanMessage, 
    AIMessage, 
    SystemMessage,
    BaseMessage
)
# Define State Schema
class CustomerSupportState(TypedDict):
    """State for customer support conversation"""
    # Conversation
    messages: Annotated[list[BaseMessage], operator.add]
    
    # Customer information
    customer_id: str
    customer_name: str
    issue_type: str
    
    # Issue details
    account_number: str
    issue_description: str
    urgency_level: str
    
    # Resolution tracking
    resolution_attempted: bool
    resolved: bool
    escalated_to_human: bool
    human_agent_notes: str
    
    # Metadata
    conversation_turns: int

print("✅ Lab 3 setup complete")

In [ ]:
# Lab 4: Define Node Functions

def ai_support_agent(state: CustomerSupportState) -> CustomerSupportState:
    """AI support agent responds to customer"""
    llm = get_llm()
    
    system_message = """You are a helpful banking customer support AI assistant.
    Your role is to:
    1. Greet customers warmly and professionally
    2. Gather necessary information about their issue
    3. Ask clarifying questions one at a time
    4. Provide solutions for common issues
    5. Escalate to human agents when needed
    
    Common issues you can help with:
    - Password resets
    - Transaction inquiries
    - Balance checks
    - Card activation
    - General account questions
    
    Escalate to human agent if:
    - Customer is frustrated or angry
    - Issue involves suspected fraud
    - Customer explicitly requests human agent
    - Issue is complex and beyond your knowledge
    
    Be concise, friendly, and professional. Ask only ONE question at a time.
    """
    
    messages = [SystemMessage(content=system_message)] + state['messages']
    
    response = llm.invoke(messages)
    
    # Determine if escalation is needed based on response
    escalate = any(phrase in response.content.lower() for phrase in [
        "transfer you to",
        "speak with a human",
        "escalate",
        "human agent"
    ])
    
    print("\n" + "="*80)
    print("🤖 AI SUPPORT AGENT")
    print("="*80)
    print(response.content)
    print("="*80 + "\n")
    
    new_state = {
        "messages": [AIMessage(content=response.content)],
        "conversation_turns": state.get('conversation_turns', 0) + 1
    }
    
    if escalate:
        new_state["escalated_to_human"] = True
    
    return new_state


def check_escalation(state: CustomerSupportState) -> Literal["get_customer_input", "escalate_to_human", "end"]:
    """Determine next step based on conversation state"""
    
    # Check if already escalated
    if state.get('escalated_to_human', False):
        return "escalate_to_human"
    
    # Check if resolved
    if state.get('resolved', False):
        return "end"
    
    # Continue conversation
    return "get_customer_input"


def get_customer_input(state: CustomerSupportState) -> CustomerSupportState:
    """
    Pause and wait for customer response
    This demonstrates multi-turn conversation with interrupt
    """
    print("\n" + "="*80)
    print("⏸️  WAITING FOR CUSTOMER RESPONSE")
    print("="*80)
    print(f"Conversation turns so far: {state.get('conversation_turns', 0)}")
    print("="*80 + "\n")
    
    # INTERRUPT to get customer input
    customer_response = interrupt({
        "message": "Waiting for customer response",
        "conversation_turn": state.get('conversation_turns', 0)
    })
    
    print("\n" + "="*80)
    print("👤 CUSTOMER RESPONSE")
    print("="*80)
    print(customer_response)
    print("="*80 + "\n")
    
    # Check if customer indicates issue is resolved
    resolved = any(phrase in customer_response.lower() for phrase in [
        "resolved",
        "fixed",
        "working now",
        "thank you",
        "that's all"
    ])
    
    return {
        "messages": [HumanMessage(content=customer_response)],
        "resolved": resolved
    }


def escalate_to_human_agent(state: CustomerSupportState) -> CustomerSupportState:
    """Escalate conversation to human agent"""
    print("\n" + "="*80)
    print("🔔 ESCALATING TO HUMAN AGENT")
    print("="*80)
    print("Conversation history and context being transferred...")
    print(f"Total turns: {state.get('conversation_turns', 0)}")
    print("="*80 + "\n")
    
    # INTERRUPT to wait for human agent
    print("\n" + "="*80)
    print("⏸️  WAITING FOR HUMAN AGENT TO JOIN")
    print("="*80 + "\n")
    
    agent_notes = interrupt({
        "message": "Human agent needed",
        "customer_id": state.get('customer_id', 'Unknown'),
        "issue_type": state.get('issue_type', 'Unknown'),
        "conversation_history": [
            {"role": msg.type, "content": msg.content} 
            for msg in state.get('messages', [])
        ]
    })
    
    print("\n" + "="*80)
    print("👨‍💼 HUMAN AGENT NOTES")
    print("="*80)
    print(agent_notes)
    print("="*80 + "\n")
    
    return {
        "human_agent_notes": agent_notes,
        "resolved": True  # Assuming human agent resolves the issue
    }

print("✅ Nodes defined")

In [ ]:
# Lab 4: Build the graph

def create_customer_support_workflow():
    """Create the customer support workflow"""
    workflow = StateGraph(CustomerSupportState)
    
    # Add nodes
    workflow.add_node("ai_agent", ai_support_agent)
    workflow.add_node("get_input", get_customer_input)
    workflow.add_node("escalate", escalate_to_human_agent)
    
    # Define edges
    workflow.add_edge(START, "ai_agent")
    
    # Conditional routing after AI agent
    workflow.add_conditional_edges(
        "ai_agent",
        check_escalation,
        {
            "get_customer_input": "get_input",
            "escalate_to_human": "escalate",
            "end": END
        }
    )
    
    # After getting customer input, go back to AI agent
    workflow.add_edge("get_input", "ai_agent")
    workflow.add_edge("escalate", END)
    
    # Compile with checkpointer
    memory = MemorySaver()
    app = workflow.compile(checkpointer=memory)
    
    return app

print("✅ Workflow created")

In [ ]:
# Lab 4: Initiate multi-turn customer support conversation

print("\n" + "="*80)
print("CUSTOMER SUPPORT WORKFLOW - MULTI-TURN CONVERSATION")
print("="*80 + "\n")

app = create_customer_support_workflow()
thread_config = {"configurable": {"thread_id": "support_session_001"}}

# Initial state with customer's first message
initial_state = {
    "messages": [
        HumanMessage(content="Hi, I'm having trouble logging into my account.")
    ],
    "customer_id": "CUST-98765",
    "customer_name": "Emily Johnson",
    "issue_type": "login_issue",
    "account_number": "",
    "issue_description": "",
    "urgency_level": "medium",
    "resolution_attempted": False,
    "resolved": False,
    "escalated_to_human": False,
    "human_agent_notes": "",
    "conversation_turns": 0
}

print("📞 CUSTOMER INITIATED SUPPORT REQUEST")
print("="*80)
print(f"Customer: {initial_state['customer_name']}")
print(f"Customer ID: {initial_state['customer_id']}")
print(f"Initial Message: {initial_state['messages'][0].content}")
print("="*80 + "\n")


In [ ]:
# Lab  4: Conversation - Turn 1

print("\n" + "🔄 TURN 1".center(80, "=") + "\n")
    
for event in app.stream(initial_state, thread_config, stream_mode="values"):
    pass

# Check if interrupted
state_1 = app.get_state(thread_config)
if state_1.tasks:
    # Customer responds
    customer_response_1 = "I keep getting an error message that says 'Invalid credentials'"
    
    print("\n▶️  Continuing conversation with customer response...\n")
    for event in app.stream(
        Command(resume=customer_response_1),
        thread_config,
        stream_mode="values"
    ):
        pass

In [ ]:
# Lab 4: Conversation - Turn 2

print("\n" + "🔄 TURN 2".center(80, "=") + "\n")
    
state_2 = app.get_state(thread_config)
if state_2.tasks:
    # Customer responds with more details
    customer_response_2 = "Yes, I tried resetting it but I'm not receiving the reset email"
    
    print("\n▶️  Continuing conversation with customer response...\n")
    for event in app.stream(
        Command(resume=customer_response_2),
        thread_config,
        stream_mode="values"
    ):
        pass

In [ ]:
# Lab 4: Conversation - Turn 3

print("\n" + "🔄 TURN 3".center(80, "=") + "\n")
    
state_3 = app.get_state(thread_config)
if state_3.tasks:
    # Customer gets frustrated and requests human agent
    customer_response_3 = "This is frustrating. Can I speak to a human agent please?"
    
    print("\n▶️  Continuing conversation with customer response...\n")
    for event in app.stream(
        Command(resume=customer_response_3),
        thread_config,
        stream_mode="values"
    ):
        pass

In [ ]:
# Lab 4 - Conversation - Human intervention & final state

state_4 = app.get_state(thread_config)
if state_4.tasks:
    # Human agent takes over
    agent_notes = """
    Spoke with Emily Johnson. Issue was:
    - Email address on file was outdated
    - Updated email to emily.j.new@email.com
    - Sent password reset link to new email
    - Customer successfully reset password
    - Issue resolved, customer satisfied
    """
    
    print("\n▶️  Human agent completing resolution...\n")
    for event in app.stream(
        Command(resume=agent_notes),
        thread_config,
        stream_mode="values"
    ):
        pass

# Final state
final_state = app.get_state(thread_config)
print("\n" + "="*80)
print("✅ SUPPORT SESSION COMPLETED")
print("="*80)
print(f"Total Conversation Turns: {final_state.values.get('conversation_turns', 0)}")
print(f"Resolved: {final_state.values.get('resolved', False)}")
print(f"Escalated: {final_state.values.get('escalated_to_human', False)}")
print("="*80 + "\n")

### Lab 4 Exercises:

1. ADD SENTIMENT ANALYSIS:
    - After each customer response, analyze sentiment
    - Auto-escalate if sentiment is negative
    - Add sentiment_score to state
    
2. IMPLEMENT CONVERSATION TIMEOUT:
    - Track time between responses
    - If customer doesn't respond for 3 turns, send follow-up
    - Add timeout_warnings to state
    
3. CREATE KNOWLEDGE BASE LOOKUP:
    - Add a node that searches knowledge base
    - Include before AI response
    - Show AI using retrieved information
    
4. BUILD CONVERSATION SUMMARY:
    - After each turn, generate a summary
    - Add conversation_summary to state
    - Use for context in next AI response
    
5. ADD CUSTOMER SATISFACTION SURVEY:
    - After resolution, ask for satisfaction rating
    - Use interrupt() to get rating
    - Store in customer_satisfaction field
    
6. IMPLEMENT MULTI-AGENT HANDOFF:
    - Create different types of human agents (billing, technical, fraud)
    - Route escalations to appropriate specialist
    - Add agent_type to escalation logic
    
7. CREATE CONVERSATION ANALYTICS:
    - Track average conversation turns
    - Calculate resolution rate
    - Measure escalation percentage
    - Add analytics_data to state
    
8. TEST DIFFERENT SCENARIOS:
    - Password reset (should resolve quickly)
    - Fraud report (should escalate immediately)
    - General inquiry (might resolve without escalation)
    - Angry customer (should escalate)